 DAEDALUS – Distributed and Automated Evolutionary Deep Architecture Learning with Unprecedented Scalability

This research code was developed as part of the research programme Open Technology Programme with project number 18373, which was financed by the Dutch Research Council (NWO), Elekta, and Ortec Logiqcare.

Project leaders: Peter A.N. Bosman, Tanja Alderliesten
Researchers: Alex Chebykin, Arthur Guijt, Vangelis Kostoulas
Main code developer: Arthur Guijt

In [ ]:
import numpy as np
import polars as pl
import matplotlib.pyplot as plt
import matplotlib as mpl
from pathlib import Path

In [ ]:
import rpy2
# Configure for notebook use.
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
%load_ext rpy2.ipython

In [ ]:
%%R
library(ggplot2)
library(dplyr)
library(patchwork)

In [ ]:
min_bound = 0.50
# retrained = pl.read_ndjson("./imagenet-a-all-evals-front.reeval-train-lr-1e-4.jsonl")
# retrained = pl.read_ndjson("./imagenet-a-all-evals-front.reeval-train-lr-1e-4-uf.jsonl")
# retrained = pl.read_ndjson("./imagenet-a-all-evals-front.reeval-train-lr-1e-5.jsonl")
# retrained = pl.read_ndjson("./imagenet-a-all-evals-front.reeval-train-lr-1e-5-uf.jsonl")

retrained = pl.read_ndjson("./imagenet-a-all-evals-front.reeval-train.jsonl")
samples = (retrained.lazy().sort("untrained_accuracy").select([
    pl.col("untrained_accuracy").alias("untrained accuracy (validation)"),
    pl.col("reeval-result").struct.field("trained_val_accuracy").alias("trained accuracy (validation)"),
    pl.col("reeval-result").struct.field("untrained_test_accuracy").alias("untrained accuracy (test)"),
    pl.col("reeval-result").struct.field("trained_test_accuracy").alias("trained accuracy (test)"),
    pl.col("untrained_loss").alias("untrained loss (validation)"),
    pl.col("reeval-result").struct.field("trained_val_loss").alias("trained loss (validation)"),
    pl.col("reeval-result").struct.field("untrained_test_loss").alias("untrained loss (test)"),
    pl.col("reeval-result").struct.field("trained_test_loss").alias("trained loss (test)"),
    pl.col("multiply-adds")
])
# -Inf out solutions that have broken completely
.with_columns([
    pl.when(pl.col("trained accuracy (validation)") < min_bound).then(-np.inf).otherwise(pl.col("trained accuracy (validation)")).alias("trained accuracy (validation)"),
    pl.when(pl.col("trained accuracy (test)") < min_bound).then(-np.inf).otherwise(pl.col("trained accuracy (test)")).alias("trained accuracy (test)"),
    pl.when(pl.col("trained loss (validation)") < min_bound).then(np.inf).otherwise(pl.col("trained loss (validation)")).alias("trained loss (validation)"),
    pl.when(pl.col("trained loss (test)") < min_bound).then(np.inf).otherwise(pl.col("trained loss (test)")).alias("trained loss (test)"),
])
).collect()
samples

In [ ]:
improvement_direction = {
    "accuracy": 1,
    "accuracy (validation)": 1,
    "accuracy (test)": 1,
    "loss": -1,
    "loss (validation)": -1,
    "loss (test)": -1,
    "loss-clip": -1,
    "total bytes": -1,
    "total_memory_bytes": -1, # dict name
    "multiply-adds": -1,
    "total_mult_adds": -1, # dict name
    # "genotype": 0, # -- not a criterion
}

In [ ]:
# Tools for computing fronts
# TODO: Modify for showing improvement?
def maybe_over(a, o):
    if len(o) == 0: return a
    else: return a.over(o)

def compute_pareto(df, group_vars, c0, c1):
    return (df.sort(c0, descending=improvement_direction[c0] > 0)
        .with_columns((pl.col(c1) * -improvement_direction[c1]).alias("c1-min"))
        .with_columns(maybe_over((pl.col("c1-min")).cummin(), group_vars).alias("mv"))
        .with_columns((maybe_over(pl.col("c1-min") < pl.col("mv").shift(1), group_vars).alias("is pareto")).fill_null(True))
        .filter(pl.col("is pareto"))
    )

def compute_2d_hv(df_pareto, ref, axis_scale, group_vars, c0, c1):
    # note - df_pareto is a df created using compute_pareto
    dhva = (df_pareto.sort(c0, descending=improvement_direction[c0] < 0)
        # Samples worse than reference point do not contribute.
        # .filter(improvement_direction[c0] * pl.col(c0) > improvement_direction[c0] * ref[0])
        # .filter(improvement_direction[c1] * pl.col(c1) > improvement_direction[c1] * ref[1])
        .with_columns(
        [
            maybe_over( improvement_direction[c0] * (pl.col(c0) - pl.col(c0).shift(1).fill_null(ref[0])) / axis_scale[0], group_vars).alias("slice_width"),
            maybe_over( improvement_direction[c1] * (pl.col(c1) - ref[1]) / axis_scale[1], group_vars).alias("slice_height"),
        ])
        .select([pl.col(group_vars), (pl.col("slice_width") * pl.col("slice_height")).alias("hv_contrib")])
        .group_by(group_vars).agg(pl.col("hv_contrib").sum()))
    return dhva

def get_transformed_front(per_run_front, grouping, c0, c1):
    # Create a dataframe for plotting in R
    pd_per_run_front = (per_run_front.lazy()
        # Add a tag so that we can track which samples were original - and which ones were added for sake
        # of continuing the lines.
        .with_columns(pl.lit(1.0).alias("is_original"))
        # For each run include an additional two rows:
        # Repeat best per objective, but replace the other objective with -Inf - as to plot towards the axes.
        .merge_sorted(per_run_front.lazy()
                    .with_columns([(pl.col(c0) * improvement_direction[c0]).alias("c0-n"),
                                    pl.lit(-np.Inf * improvement_direction[c1]).alias(c1),
                                    pl.lit(0.0).alias("is_original")])
                    .group_by(grouping, maintain_order=True)
                    .agg(pl.all().sort_by("c0-n").last())
                    .select(per_run_front.columns + ["is_original"]), "file")
        .merge_sorted(per_run_front.lazy()
                    .with_columns([(pl.col(c1) * improvement_direction[c1]).alias("c1-n"),
                                    pl.lit(-np.Inf * improvement_direction[c0]).alias(c0),
                                    pl.lit(0.0).alias("is_original")])
                    .group_by(grouping, maintain_order=True)
                    .agg(pl.all().sort_by("c1-n").last())
                    .select(per_run_front.columns + ["is_original"]), "file")
        # Add c0 and c1 as a named column
        .with_columns([pl.col(c0).alias("c0"), pl.col(c1).alias("c1")])
        # Sort, for good measure
        .sort(grouping + [c0, c1])
        # Collect & convert to pandas in order to transfer.
        .collect().to_pandas())
    return pd_per_run_front

In [ ]:
samples_pd = samples.to_pandas()

In [ ]:
%%R
library(ggplot2)

In [ ]:
%%R -i samples_pd -w 500 -h 270

scientific_10 <- function(x) {   parse(text=gsub("e\\+*", " %*% 10^", scales::scientific_format()(x))) } 

pla <- ggplot(samples_pd, aes(y = `multiply-adds`, yend = `multiply-adds`)) +
    geom_segment(aes(x = `untrained accuracy (validation)`, xend=`trained accuracy (validation)`), arrow = arrow(length = unit(0.02, "npc")), alpha=0.3) +
    geom_point(aes(x = `untrained accuracy (validation)`), alpha=0.3) +
    geom_point(aes(x = `trained accuracy (validation)`), color="blue", shape = 21, fill=NA) +
    scale_y_continuous(label=scientific_10) +
    labs(x = "accuracy (validation)") +
    theme_bw() +
    theme(
      legend.position="bottom",
      axis.text.x = element_text(angle = 45, vjust = 1, hjust=1),
      plot.background = element_rect(fill='transparent', color=NA),
      strip.background = element_blank())

plb <- ggplot(samples_pd, aes(y = `multiply-adds`, yend = `multiply-adds`)) +
    geom_segment(aes(x = `untrained accuracy (test)`, xend=`trained accuracy (test)`), arrow = arrow(length = unit(0.02, "npc")), alpha=0.3) +
    geom_point(aes(x = `untrained accuracy (test)`), alpha=0.3) +
    geom_point(aes(x = `trained accuracy (test)`), color="blue", shape = 21, fill=NA) +
    scale_y_continuous(label=scientific_10) +
    labs(x = "accuracy (test)") +
    theme_bw() +
    theme(
      legend.position="bottom",
      axis.text.y=element_blank(),
      axis.title.y=element_blank(),
      axis.text.x = element_text(angle = 45, vjust = 1, hjust=1),
      plot.background = element_rect(fill='transparent', color=NA),
      strip.background = element_blank())

plt <- (pla | plb)# + plot_layout(guides = "collect") & theme(legend.position="bottom")
# ggsave("result-retrain.pdf")
plt